<img src = 'fotos/logo_dani.jpeg'>

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX 
import itertools
import pickle

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from catboost import CatBoostRegressor

from IPython.display import clear_output
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
def rmse(real, predicted):
    return np.sqrt(((real - predicted) ** 2).mean())

## Ficheros y rutas de entrada/salida 

In [3]:
dir_in = '../../datos/datos_desarrollo'
file1_in = 'consumo_final.csv'
dir_out = dir_in

## Primera estrategia de modelaje 

### Carga de datos 

In [4]:
df_consumo = pd.read_csv(os.path.join(dir_in, file1_in), sep = ';')
df_consumo.columns = [columna.lower() for columna in df_consumo.columns]
df_consumo.rename(columns = {'fecha_inicio': 'mes_inicio_temp', 'fecha_fin': 'mes_fin_temp'}, inplace = True)
df_consumo.date = pd.to_datetime(df_consumo.date, format = '%Y-%m-%d')
precio_model = df_consumo[['ccaa', 'producto', 'volumen_miles_de_kg', 'valor_miles_de_€', 'precio_medio_kg', 'date']]

In [5]:
cmpl_model_dict = {product: {comunidad: precio_model[(precio_model.producto.eq(product))&
                                                      (precio_model.ccaa.eq(comunidad))].drop(['producto', 'ccaa'], axis = 1)
                              for comunidad in precio_model.ccaa.unique()} for product in precio_model.producto.unique()}

In [6]:
df_prueba = df_consumo[(df_consumo.ccaa.isin(['Andalucia', 'Aragon'])) & (df_consumo.producto.isin(['Patatas', 'Mango']))]
prueba_model = df_prueba[['ccaa', 'producto', 'volumen_miles_de_kg', 'valor_miles_de_€', 'precio_medio_kg', 'date']]
sample_model_dict = {product: {comunidad: prueba_model[(prueba_model.producto.eq(product))&
                                                      (prueba_model.ccaa.eq(comunidad))].drop(['producto', 'ccaa'], axis = 1)
                              for comunidad in prueba_model.ccaa.unique()} for product in prueba_model.producto.unique()}

In [ ]:
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

### Modelaje 

In [ ]:
def best_sarima(variable, model_dict):
    min_error_df = pd.DataFrame(columns = ['comunidad', 'producto', 'error'])
    try:
        len(predicciones)
    except NameError:
        predicciones = pd.DataFrame(columns = ['ccaa', 'producto'])
    vuelta_general = 1
    fila_df = 0
    for producto, v in model_dict.items():
        n_comunidad = 1
        for comunidad in v.keys():
            combinacion = 1
            error_dict = {}
            timeseries_data = model_dict[producto][comunidad][[variable, 'date']].copy()
            timeseries_data.index = pd.DatetimeIndex(timeseries_data.date)
            timeseries_data.drop('date', axis = 1, inplace = True)
            timeseries_data.sort_index(inplace = True)

            train = timeseries_data.loc[timeseries_data.index <= '2020-02-01']
            test = timeseries_data.loc[timeseries_data.index > '2020-02-01']
            
            for param in pdq:
                for seasonal_param in seasonal_pdq:
                    print('Tienes', len(model_dict), 'productos y vas por el', vuelta_general, '. Este producto tiene', 
                          len(v), 'comunidades y vas por la', n_comunidad, 'Hay un total de', len(pdq) * len(seasonal_pdq), 
                          'combinaciones y vas por la', combinacion, '. La variable es', variable)
                    try:
                        print(param, seasonal_param)
                        mod = SARIMAX(train, 
                            order = param,
                            seasonal_order = seasonal_param, 
                            enforce_stationarity = False,
                            enforce_invertibility = False)                
                        result = mod.fit()
                        ypred = result.get_forecast(steps = len(test)).predicted_mean.values
                        error = rmse(test.values, ypred)
                        error_dict.update({error: [param, seasonal_param]})
                    except IndexError:
                        continue
                    combinacion += 1
                    clear_output(wait = True)
                    
            min_error = min(list(error_dict.keys()))
            min_error_df.loc[fila_df, ['comunidad', 'producto', 'error']] = comunidad, producto, min_error
            final_model = SARIMAX(train, order = error_dict[min_error][0], seasonal_order = error_dict[min_error][1], 
                                  enforce_stationarity = False, enforce_invertibilty = False)
            final_result = final_model.fit()
            final_ypred = final_result.get_forecast(steps = len(test))
            ypred_df = pd.DataFrame(final_ypred.predicted_mean).rename(columns = {'predicted_mean': str(variable) + '_predicted'})
            pred_ci = final_ypred.conf_int()
            pred_ci_df = pd.concat([ypred_df, pred_ci], axis = 1)
            pred_ci_df[['ccaa', 'producto']] = comunidad, producto
            
            line = pd.to_datetime('2020-02-01', format = '%Y-%m-%d')
            real_value = train.loc[line].values[0]
            new_row =  pd.DataFrame({'ccaa': comunidad, 'producto': producto, (variable + '_predicted'): real_value, 
                         ('lower ' + variable):real_value, ('upper ' + variable): real_value}, 
                                    index = [line])
            predicciones = pd.concat([predicciones, new_row, pred_ci_df], axis = 0)
            n_comunidad += 1
            fila_df += 1
        vuelta_general += 1
    return predicciones, min_error_df

In [ ]:
df_peque, error_peque = best_sarima('volumen_miles_de_kg', cmpl_model_dict)

with open('df_peque.pkl', 'wb') as fp:
    pickle.dump(df_peque, fp)    
with open('error_peque.pkl', 'wb') as fp:
    pickle.dump(error_peque, fp)

In [ ]:
with open('df_peque.pkl', 'rb') as fp:
    df_peque = pickle.load(fp)
with open('error_peque.pkl', 'rb') as fp:
    error_peque = pickle.load(fp)

In [ ]:
df_peque2, error_peque2 = best_sarima('valor_miles_de_€', cmpl_model_dict)
df_medi = pd.concat([df_peque, df_peque2], axis = 1)
error_medi = pd.concat([error_peque, error_peque2], axis = 1)
with open('df_medi.pkl', 'wb') as fp:
    pickle.dump(df_medi, fp)    
with open('error_medi.pkl', 'wb') as fp:
    pickle.dump(error_medi, fp)

In [ ]:
with open('df_medi.pkl', 'rb') as fp:
    df_medi = pickle.load(fp)
with open('error_medi.pkl', 'rb') as fp:
    error_medi = pickle.load(fp)

In [ ]:
df_peque3, error_peque3 = best_sarima('precio_medio_kg', cmpl_model_dict)
df_gran = pd.concat([df_medi, df_peque3], axis = 1)
error_gran = pd.concat([error_medi, error_peque3], axis = 1)
with open('df_gran.pkl', 'wb') as fp:
    pickle.dump(df_gran, fp)    
with open('error_gran.pkl', 'wb') as fp:
    pickle.dump(error_gran, fp)

In [ ]:
with open('df_gran.pkl', 'rb') as fp:
    df_gran = pickle.load(fp)
with open('error_gran.pkl', 'rb') as fp:
    error_gran = pickle.load(fp)

In [ ]:
df_final = df_gran.T.drop_duplicates().T
df_final = df_final.loc[:, 'producto':]
df_final.to_csv(os.path.join(dir_out, 'predicciones_consumo.csv'), sep = ';')

## Segunda estrategia de modelaje 

### Preparación de los datos 

In [7]:
df_consumo = pd.read_csv(os.path.join(dir_in, file1_in), sep = ';')
df_consumo.columns = [columna.lower() for columna in df_consumo.columns]
df_consumo.rename(columns = {'fecha_inicio': 'mes_inicio_temp', 'fecha_fin': 'mes_fin_temp'}, inplace = True)
df_consumo.date = pd.to_datetime(df_consumo.date, format = '%Y-%m-%d')
precio_model = df_consumo[['ccaa', 'producto', 'volumen_miles_de_kg', 'valor_miles_de_€', 'precio_medio_kg', 'date']]

In [10]:
product_key_dict = {producto: precio_model[precio_model.producto == producto] for producto in precio_model.producto.unique()}

In [17]:
prueba = product_key_dict['Patatas']
train = prueba[prueba.date < '2020-01-01']
val = prueba[(prueba.date >= '2020-01-01') & (prueba.date <'2020-03-01')]

In [21]:
train_var = train[['precio_medio_kg', 'date', 'ccaa']]
train_var.index = pd.DatetimeIndex(train_var.date)
train_var.drop('date', axis = 1, inplace = True)
train_var.sort_index(inplace = True)

In [ ]:
mod = SARIMAX(train, 
            order = param,
            seasonal_order = seasonal_param, 
            enforce_stationarity = False,
            enforce_invertibility = False)                
result = mod.fit()
ypred = result.get_forecast(steps = len(test)).predicted_mean.values
error = rmse(test.values, ypred)
error_dict.update({error: [param, seasonal_param]})

In [29]:
train_var_spread = pd.pivot_table(train_var, index = train_var.index, columns = 'ccaa', values = 'precio_medio_kg')

In [32]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=6
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(train_var_spread, variables = train_var_spread.columns)

In [44]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(train_var_spread)

LinAlgError: Matrix is not positive definite

In [46]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.") 

In [48]:
from statsmodels.tsa.stattools import adfuller

In [49]:
for name, column in train_var_spread.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "Andalucia" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2.1045
 No. Lags Chosen       = 0
 Critical value 1%     = -3.753
 Critical value 5%     = -2.998
 Critical value 10%    = -2.639
 => P-Value = 0.2427. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "Aragon" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -1.6211
 No. Lags Chosen       = 1
 Critical value 1%     = -3.77
 Critical value 5%     = -3.005
 Critical value 10%    = -2.643
 => P-Value = 0.4722. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "Asturias" 
    -----------------------------------------------
 Null Hypothesis: Data h

In [54]:
df_differenced = train_var_spread.diff().dropna()
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "Andalucia" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2.1332
 No. Lags Chosen       = 9
 Critical value 1%     = -4.069
 Critical value 5%     = -3.127
 Critical value 10%    = -2.702
 => P-Value = 0.2314. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "Aragon" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -7.641
 No. Lags Chosen       = 0
 Critical value 1%     = -3.77
 Critical value 5%     = -3.005
 Critical value 10%    = -2.643
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "Asturias" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationa

In [59]:
from statsmodels.tsa.vector_ar.var_model import VAR

In [62]:
model = VAR(df_differenced)
for i in [1,2,3,4,5,6,7,8,9]:
    result = model.fit(maxlags = 1)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1


LinAlgError: 5-th leading minor of the array is not positive definite